In [1]:
N_JOBS = 7
N_MACHINES = 4

In [2]:
import random

def generateData(seed=0):
    PROC = [ [ 0 for m in range(N_MACHINES) ] for j in range(N_JOBS) ]
    random.seed(seed)
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            PROC[j][m] = random.randint(1,9)
    return PROC

In [3]:
PROC = generateData(2)

In [4]:
PROC

[[1, 2, 2, 6],
 [3, 5, 5, 4],
 [1, 3, 7, 7],
 [9, 6, 9, 8],
 [9, 5, 1, 1],
 [6, 8, 6, 7],
 [7, 9, 3, 9]]

In [5]:
import pulp

In [6]:
def val(x):
    return int(pulp.value(x))

In [7]:
prob = pulp.LpProblem("JobScheduling",pulp.LpMinimize)

`J[j]` defines the sequencing of jobs 'j' and is just a permutation of the job number:

In [8]:
JJ = pulp.LpVariable.dicts("J", (range(N_JOBS), range(N_JOBS)),
                           lowBound=0, upBound=1, cat='Integer')

In [9]:
def job(n):
    for j in range(N_JOBS):
        if val(JJ[n][j])==1:
            return j

In [10]:
def jobSequence():
    acc = []
    for j in range(N_JOBS):
        for jj in range(N_JOBS):
            if pulp.value(JJ[j][jj])==1:
                acc.append(jj)
    return acc

In [11]:
def proc(job, machine):
    global PROC
    return pulp.lpSum( [ PROC[j][machine] * JJ[j][job] for j in range(N_JOBS) ] ) 

`WAIT[j][m]` describes the wait time of job `j` before machine `m`

In [12]:
Wait = pulp.LpVariable.dicts("WAIT", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

`IDLE[j][m]` describes the idle time of machine `m` before processing job `j` 

In [13]:
Idle = pulp.LpVariable.dicts("IDLE", (range(N_JOBS), range(N_MACHINES)),
                             lowBound=0, cat='Integer')

`START[j][m]` describes the start time of machine `m` processing job `j`

In [14]:
Start = pulp.LpVariable.dicts("START", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

`STOP[j][m]` describes the stop time of machine `m` after processing job `j`

In [15]:
Stop = pulp.LpVariable.dicts("STOP", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

In [16]:
prob += Stop[N_JOBS-1][N_MACHINES-1]

In [17]:
for j in range(N_JOBS):
    prob += pulp.lpSum( [ JJ[j][jj] for jj in range(N_JOBS) ] ) == 1
    prob += pulp.lpSum( [ JJ[jj][j] for jj in range(N_JOBS) ] ) == 1

In [18]:
for m in range(N_MACHINES):
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ Idle[ji][m] + proc(ji, m) for ji in range(j) ] ) + Idle[j][m] == Start[j][m]

In [19]:
for m in range(N_MACHINES):
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ Wait[j][mi] + proc(j, mi) for mi in range(m) ] ) + Wait[j][m] == Start[j][m]

In [20]:
for j in range(N_JOBS):
    for m in range(N_MACHINES):
        prob += Start[j][m] + proc(j,m) == Stop[j][m]

In [21]:
solvers = pulp.listSolvers(onlyAvailable=True) 
solver = pulp.getSolver(solvers[0], msg=0)
prob.solve(solver)

1

In [22]:
def schedule():
    row = ""
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            |" \
                f" Machine: {m:1d} |"
    row += '\n'
    for j in range(N_JOBS):
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Idle: {val(Idle[j][m]):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Start:{val(Start[j][m]):4d} |"  
        row += '\n' 
        row += f'| Job: {job(j):2d} |'
        for m in range(N_MACHINES):
            row += f" Wait: {val(Wait[j][m]):4d} | " \
                   f"Proc: {val(proc(j,m)):4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Stop: {val(Stop[j][m]):4d} |"  
        row += '\n' 
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    return row

The processing times per job and machine:

In [23]:
PROC

[[1, 2, 2, 6],
 [3, 5, 5, 4],
 [1, 3, 7, 7],
 [9, 6, 9, 8],
 [9, 5, 1, 1],
 [6, 8, 6, 7],
 [7, 9, 3, 9]]

The optimal job schedule:

In [24]:
print(jobSequence())

[0, 2, 1, 4, 6, 5, 3]


The processing time of the optimal job schedule:

In [25]:
int(pulp.value(prob.objective))

52

The optimal job schedule in detail:

In [26]:
print(schedule())

|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    1 |            | Idle:    3 |            | Idle:    5 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   1 |            | Start:   3 |            | Start:   5 |
| Job:  0 | Wait:    0 | Proc:    1 | Wait:    0 | Proc:    2 | Wait:    0 | Proc:    2 | Wait:    0 | Proc:    6 |
|         |            | Stop:    1 |            | Stop:    3 |            | Stop:    5 |            | Stop:   11 |
|---------|------------|------------|------------|------------|---------